In [27]:
import numpy as np
import pandas as pd 
import csv
from sklearn.metrics import f1_score

In [104]:
data_root = '/Users/svloaner/Desktop/yelp/'

cluster = "pca500"
#pca500 pca kpca_cosine pca_mle

train_photo_to_biz = pd.read_csv(data_root+'train_photo_to_biz_ids.csv')
train_df = pd.read_csv(data_root+ 'train_feature_' + cluster + '.csv')
train_df1 = pd.merge(train_photo_to_biz, train_df, how='inner', on='photo_id')
train_df2 = train_df1.groupby('business_id')


In [105]:
feature_column = train_df.columns[:-1]

out_colum_name = ['business_id']
out_colum_name.extend(feature_column.values)

def getOneRow(name, grouped_df, train=1):
    out_list = [name]
    for f in feature_column:
        avg = np.mean(grouped_df[f])
        out_list.append(avg)        
    return out_list

In [106]:
train_out_handle = open('train_business_feature2.csv', "w")
train_writer = csv.writer(train_out_handle)
train_writer.writerow(out_colum_name)
counter = 0
for name, group in train_df2:
    out_row = getOneRow(name, group, train=1)
    assert len(out_row) == len(out_colum_name)
    train_writer.writerow(out_row)
    counter += 1
    if counter%10000 == 0:
        print counter
train_out_handle.close()

In [107]:
train_business_feature = pd.read_csv(data_root+'train_business_feature2.csv')
train_label = pd.read_csv(data_root+'train.csv')
train_df3 = pd.merge(train_business_feature, train_label, how='inner', on='business_id')

In [108]:
train_df3['labels'].fillna('', inplace=True)
y_train = np.array([y.split() for y in train_df3['labels']])
y_train = [map(int, y) for y in y_train]

In [109]:
train_business_feature.drop('business_id', axis=1, inplace=True)

In [110]:
from sklearn import svm, datasets
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
import multiprocessing

cores=multiprocessing.cpu_count()-2

mlb = MultiLabelBinarizer()
y_ptrain_mlb= mlb.fit_transform(y_train) 

random_state = np.random.RandomState(0)

X_ptrain, X_ptest, y_ptrain, y_ptest = train_test_split(train_business_feature, y_ptrain_mlb, 
                 test_size=.2,random_state=random_state)
print X_ptrain.shape, X_ptest.shape, y_ptrain.shape, y_ptest.shape

(1600, 500) (400, 500) (1600, 9) (400, 9)


In [114]:
clf_svm = svm.SVC(kernel='rbf', C= 2.0, probability=True)
from sklearn.ensemble import BaggingClassifier
bagging = BaggingClassifier(clf_svm,
                               max_samples=0.8, max_features=0.5)

classifier = OneVsRestClassifier(bagging, n_jobs=cores)

classifier.fit(X_ptrain, y_ptrain)

OneVsRestClassifier(estimator=BaggingClassifier(base_estimator=SVC(C=2.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
         bootstrap=True, bootstrap_features=False, max_features=0.5,
         max_samples=0.8, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False),
          n_jobs=6)

In [115]:
y_ppredict = classifier.predict(X_ptest)

In [116]:
from sklearn.metrics import f1_score

print "F1 score: ", f1_score(y_ptest, y_ppredict, average='micro') 
print "Individual Class F1 score: ", f1_score(y_ptest, y_ppredict, average=None)

F1 score:  0.851407861723
Individual Class F1 score:  [ 0.72173913  0.85388128  0.89438202  0.67506297  0.80165289  0.91181989
  0.9527027   0.78927203  0.88641425]


In [72]:
#########test
test_photo_to_biz = pd.read_csv(data_root+'test_photo_to_biz.csv')
test_df = pd.read_csv(data_root+ 'test_feature_'  + cluster + '.csv')
test_df1 = pd.merge(test_photo_to_biz, test_df, how='inner', on='photo_id')
test_df2 = test_df1.groupby('business_id')
feature_column = test_df.columns[:-1]
test_out_handle = open('test_business_feature2.csv', "w")
test_writer = csv.writer(test_out_handle)
test_writer.writerow(out_colum_name)
counter = 0
for name, group in test_df2:
    out_row = getOneRow(name, group, train=0)
    assert len(out_row) == len(out_colum_name)
    test_writer.writerow(out_row)
    counter += 1
    if counter%10000 == 0:
        print counter
test_out_handle.close()

##################

10000


In [73]:
classifier.fit(train_business_feature, y_ptrain_mlb)

test_business_feature = pd.read_csv(data_root+'test_business_feature2.csv')
business_id = test_business_feature['business_id'].reshape(-1,1)
test_business_feature.drop('business_id', axis=1, inplace=True)
y_predict_test = classifier.predict(test_business_feature)

In [74]:
y_predict_label = mlb.inverse_transform(y_predict_test)

df = pd.DataFrame(columns=['business_id','labels'])

for i in range(len(y_predict_label)):
    biz = business_id[i][0]
    label = y_predict_label[i]
    label = str(label)[1:-1].replace(",", " ")
    df.loc[i] = [str(biz), label]

with open(data_root+"submission3.csv",'w') as f:
    df.to_csv(f, index=False) 